In [3]:
import shutil

# Original dataset path
original_dataset_path = "/kaggle/input/medqa-senior/dataset.txt"

# Destination path in the working directory
destination_path = "/kaggle/working/dataset_copy.txt"

# Copy the contents of the original dataset file to the destination file
shutil.copyfile(original_dataset_path, destination_path)

# Now you can use the copied dataset file for training
dataset_path = destination_path


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2-medium"  # or any other pre-trained model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load domain-specific dataset
print(dataset_path)
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=dataset_path,
    block_size=128  # Adjust block size according to your dataset
)

# Prepare data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust number of epochs as needed
    per_device_train_batch_size=4,  # Adjust batch size as needed
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Fine-tune the model on the domain-specific dataset
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")



/kaggle/working/dataset_copy.txt


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import textwrap

# Load fine-tuned GPT-2 model and tokenizer
model_path = "./fine_tuned_model"  # Path to your fine-tuned model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained(model_path)

# Define a function to generate responses to questions
def generate_response(question, max_length=50):
    input_text = question
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage
question = "What is Alzheimer's disease?"
response = generate_response(question)
response = response[:response.rfind('.')+1]
print("Response:", textwrap.fill(response,width = 80))
